# Check Darcy/Forcheimer source terms in VDF/VEF

In [ ]:
from trustutils import run
run.introduction("Pierre LEDAC (CS)")

### Description

Solve a Darcy flow in a conic geometry. \
Velocity field (0.2m/s) is imposed at the inlet (smallest area). \
Pressure field (0 Pa) is imposed at the outlet (buggest area). \
Solve dU/dt=-grad(P/rho)-nu.U/K-Cf.U|U|/sqrt(K1) \
by using Navier Stokes equation and neglecting convection and diffusion terms. \
The first source term is Darcy term and the second one, the inertial Forchheimer term. \
See http://en.wikipedia.org/wiki/Darcy's_law \
At the convergence, we have grad(P/rho)=nu.U/K+Cf.U|U|/sqrt(K1) \
The time step is set to 0.001s and should be carefully cherry picked to avoid non convergence.

In [ ]:
run.TRUST_parameters()

In [ ]:
from trustutils import run
for i in ["VDF", "QC_VDF", "VEF", "QC_VEF"]:
    run.addCase(".",f"DarcyFlow_{i}.data")

run.printCases()
run.runCases()

## 3D results



### Pressure (Incompressible model, VDF discretisation)

In [ ]:
from trustutils import visit 
visit.showField("./DarcyFlow_VDF.lata","Pseudocolor","PRESSION_PA_ELEM_dom")

### Pressure (Quasi compressible model, VDF discretisation)

In [ ]:
from trustutils import visit 
visit.showField("./DarcyFlow_QC_VDF.lata","Pseudocolor","PRESSION_PA_ELEM_dom") 

### Pressure (Incompressible model, VEF discretisation)

In [ ]:
from trustutils import visit 
visit.showField("./DarcyFlow_VEF.lata","Pseudocolor","PRESSION_PA_ELEM_dom") 

### Pressure (Quasi compressible model, VEF discretisation)

In [ ]:
from trustutils import visit 
visit.showField("./DarcyFlow_QC_VEF.lata","Pseudocolor","PRESSION_PA_ELEM_dom") 

## 1D results

### Convergence of the axial velocity

In [ ]:
from trustutils.jupyter import plot

graphique=plot.Graph("Plot") 

graphique.addPoint('./DarcyFlow_VDF_SONDE_VITESSE.son'   ,marker="-|",color='r'     ,compo=1 ,label="Incompressible model, VDF discretisation")
graphique.addPoint('./DarcyFlow_QC_VDF_SONDE_VITESSE.son',marker="-x",color='g'     ,compo=1 ,label="Quasi compressible model, VDF discretisation")

graphique.addPoint('./DarcyFlow_VEF_SONDE_VITESSE.son'   ,marker="-v",color='b'     ,compo=1,label="Incompressible model, VEF discretisation")
graphique.addPoint('./DarcyFlow_QC_VEF_SONDE_VITESSE.son',marker="-^",color='violet',compo=1,label="Quasi compressible model, VEF discretisation")

### Pressure profile

In [ ]:
from trustutils import plot 

a = plot.Graph('Pressure profile')

x=plot.loadText("./DarcyFlow_VDF_SEGMENT_PRESSION.coupe")
a.add(x[0],x[1],"-|",label="Incompressible model, VDF discretisation",color='r')

x=plot.loadText("./DarcyFlow_QC_VDF_SEGMENT_PRESSION.coupe")
a.add(x[0],x[1],"-x",label="Quasi compressible model, VDF discretisation",color='g' )

x=plot.loadText("./DarcyFlow_VEF_SEGMENT_PRESSION.coupe")
a.add(x[0],x[1],"-v",label="Incompressible model, VEF discretisation",color='b' )

x=plot.loadText("./DarcyFlow_QC_VEF_SEGMENT_PRESSION.coupe")
a.add(x[0],x[1],"-^",label="Quasi compressible model, VEF discretisation",color='violet')



## Conclusion

I) Isotherm flow has been specified for the quasi compressible model \
so the results should match whith the incompressible model. \
II) The more refined the mesh, the smaller the VDF/VEF differences for pressure and velocity fields.


## Data Files

### DarcyFlow_VDF

In [ ]:
from trustutils import run 
run.dumpDataset("./DarcyFlow_VDF.data") 

### DarcyFlow_QC_VDF

In [ ]:
from trustutils import run 
run.dumpDataset("./DarcyFlow_QC_VDF.data") 

### DarcyFlow_VEF

In [ ]:
from trustutils import run 
run.dumpDataset("./DarcyFlow_VEF.data") 

### DarcyFlow_QC_VEF

In [ ]:
from trustutils import run 
run.dumpDataset("./DarcyFlow_QC_VEF.data") 